<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек-и-описание-функций" data-toc-modified-id="Импорт-библиотек-и-описание-функций-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек и описание функций</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Мешок-слов" data-toc-modified-id="Мешок-слов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Мешок слов</a></span></li><li><span><a href="#Би-граммы" data-toc-modified-id="Би-граммы-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Би-граммы</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорт библиотек и описание функций

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from IPython.display import HTML, display
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import nltk
from tqdm.auto import tqdm
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn  as sns
from sklearn.model_selection import cross_val_score
import time

# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import StandardScaler 
# from sklearn.feature_extraction.text import CountVectorizer 



In [2]:
#параметры
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 300

#Параметры для отладки
DEBUGING = False               #вывод на экран текущих параметров

models_params_find = False

R_STATE = 737
N_JOBS = -1  
filled_VehicleType=False

MODEL_TEXT_NAME = {'LinearReg':'Линейная регрессия'
                ,'LogisticReg':'Логическая регрессия'
                    ,'DecTreeReg':'Дерево принятия решений для регрессии' 
                    ,'RndForestReg':'Случайный лес для регрессии'
                   ,'DecTreeCls':'Дерево принятия решений для классификации' 
                    ,'RndForestCls':'Случайный лес для классификации'
                  ,'LGBMReg':'Ансамбль LightGBM для регрессии'
                  ,'LGBMCls':'Ансамбль LightGBM для классификации'}

In [3]:
def disp_font_size(text,font_size=3):
    display(HTML(f"<font size='{font_size}'>{text}</font>"))
def info_print (data, title, column='', bins=20, unit=''
                , prntInfo=True, prntGraph=True, prnt_smpl=True
               , returnMetric=False, drop_out=False):
    '''
    функция для отоброжения информации о наборе данных
    '''
    div = ''.join(['=' for i in range(1,len(title))])
    print(div)
    print('\n',title,'\n')
    print(div)
    if prntInfo:        
        data.info()
        print(div)
        print(f'Количество дубликатов: {data.duplicated().sum()}')
        print(div)
        if data.duplicated().sum() > 0:
            try:
                print(f'Пример дубликатов:')
                display(data.loc[data.duplicated(keep=False)].sort_values(by=data.columns[1]).head())            
                print(div)
            except: None    
        if prnt_smpl:
            print(f'Пример первых 5 записей:')
            display(data.head())             
            print(div)
     
    if column != '':
        data_prin = data[column]
    else:
        data_prin = data
        
    if prntGraph:        
        #try:
            std_dev = np.std(data_prin) #Стандартное распределение
            mu = data_prin.mean() #среднее значение 
            sigma3_min = mu - 3*std_dev
            sigma3_max = mu + 3*std_dev
            print(f'Среднее: {round(mu,2)} \n')
            print(f'Стандартное отклонение: {round(std_dev,2)} \n')
            print(f'Доверительный интервал: от {round(sigma3_min,2)} до {round(sigma3_max,2)} \n')    
            print(f'Минимум: {round(data_prin.min(),4)} Максимум: {round(data_prin.max(),4)} \n')    
            fig, ax = plt.subplots(1,2,figsize=(17, 3))
            fig.suptitle(title) 
            sns.boxplot(data_prin, ax=ax[0])
         
            sns.distplot(data_prin, ax=ax[1],bins=bins)              
            #границы по правилам трёх сигм
            plt.axvline(sigma3_min,color='r',linestyle='--') 
            plt.axvline(sigma3_max,color='r',linestyle='--')
            ax[0].set_xlabel(unit, fontsize=15, color='black')            
            ax[1].set_xlabel(unit, fontsize=15, color='black')
            plt.show()      
            if returnMetric:
                if drop_out: 
                    mu = data_prin.loc[(sigma3_min < data_prin) & (data_prin < sigma3_max)].mean()
                    print(f'Среднее без учёта выбросов: {round(mu,2)} \n')                    
                return mu, std_dev
            
        #except: None

In [4]:
def display_metric(score, scorer, title=''): 
    """
    Функция для вывода метрик
    """                   
    scorer_name = re.sub(r".*\(|\,.*|\)", '', str(scorer))

    disp_font_size(f" {title} {scorer_name.upper()}: = {score}",5)  

In [5]:
def score_cv(model, features, target, scorer, cv=5, mean=True):
    el_time = time.time()
    result = cross_val_score(model, features, target, scoring=scorer, cv=cv, n_jobs=N_JOBS) 

    el_time = round(time.time() - el_time,2)
    if mean: result = round(result.mean(),4)
    return result, el_time 

def  make_pipe(model, **kwargs):
    return Pipeline([
    ('vector', TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 1))),
    ('clf', model)
    ])
def make_clf(model_type, **kwargs):
    if model_type == 'RndForestReg':
        model = make_pipe(RandomForestRegressor(random_state=R_STATE, **kwargs))
    elif model_type == 'DecTreeReg':        
        model = make_pipe(DecisionTreeRegressor(random_state=R_STATE, **kwargs))
    elif model_type == 'LogisticReg':
        model = make_pipe(LogisticRegression(**kwargs))
    elif model_type == 'RndForestCls':
        model = make_pipe(RandomForestClassifier(random_state=R_STATE, **kwargs))
    elif model_type == 'DecTreeCls':        
        model = make_pipe(DecisionTreeClassifier(random_state=R_STATE, **kwargs))
    elif model_type == 'LGBMCls':
        model = make_pipe(LGBMClassifier(boosting_type='gbdt',random_state=R_STATE, **kwargs))       
    else: return 0, 'Тип модели не найден'
    return model

#Таблица для хранения лучших результатов моделей
columns_model = ['model','g_param','Score']
models_valid = pd.DataFrame(columns=columns_model)    

In [6]:
def get_model(params, train_features, train_target):    
    model = make_clf(params['model'], **params['g_param'])
    pbar = tqdm(total = 1)
    model.fit(train_features, train_target)   
    pbar.update(1)    
    pbar.close()        
    return model


def get_param_list(kwargs):
    params_list = pd.DataFrame()    
    if len(kwargs) > 0:        
        new_str = pd.Series()
        for key, param in kwargs.items():
            old_param_list = params_list.copy()
            params_list = pd.DataFrame()
            num_params = len(old_param_list)
            if num_params < 1 : num_params=1
            for str_ind in range(num_params):
                if isinstance(param, range) | isinstance(param, np.ndarray): 
                    for p in param:
                        try:
                            new_str = old_param_list.iloc[str_ind]
                        except: None
                        new_str[key] = p
                        params_list = params_list.append(new_str, ignore_index=True)                    
                else:
                    try:
                        new_str = old_param_list.iloc[str_ind]
                    except: None
                    new_str[key] = param
                    params_list = params_list.append(new_str, ignore_index=True)
    if len(params_list) < 1: params_list.loc[0,'no_param']='no_param'
    return params_list

    
def get_model_params(model_type, features, target, scorer, title=''
                 , disp_metr=True
                 ,returnModel=False, debug=False, **kwargs):

    """
    Функция для создания модели  
    
    """
    
    columns_model = ['model','g_param','el_time','score']
    models_param = pd.DataFrame(columns=columns_model)    

       
    print('==========================')    
    disp_font_size(f"Модель: {MODEL_TEXT_NAME[model_type]} {title} <br>",5)
    
    params_list = get_param_list(kwargs)

    pbar = tqdm(total = len(params_list))
    params = {}
    for ind, row in params_list.iterrows():
        for col_n, val in row.items():
            if col_n != 'no_param':
                if isinstance(val, str): params[col_n] = val
                elif float(val)%1>0: params[col_n] = float(val)
                else: params[col_n] = int(val)      
        model = make_clf(model_type, **params)  
        try:
            score, el_time = score_cv(model, features, target, scorer)            
            if debug: print(f'{kwargs}|{score}')              
            models_param = models_param.append({'model': model_type
                                               ,'g_param':{**params}                                                
                                               ,'el_time':el_time
                                               ,'score':score}
                                               , ignore_index=True)   
        except:
            models_param = models_param.append({'model': model_type
                                               ,'g_param':{**params}                                                
                                               ,'el_time':'Err'
                                               ,'score':'Err'}
                                               , ignore_index=True)   

        pbar.update(1)      
    pbar.close()        
    #Вывод результатов    
    models_param = models_param.loc[~models_param['score'].isin(['Err','-'] )]
    models_param = models_param.sort_values(by='score', ascending=False).reset_index(drop=True)
 
    
    if disp_metr: 
        display(models_param)
        disp_font_size(re.sub(r'[\{\}]', '', str(models_param.loc[0,'g_param'])),5)
        disp_font_size(f"Время: {models_param.loc[0,'el_time']}",5)
        display_metric(models_param.loc[0,'score'], scorer)             
        
  
    return models_param

def models_test(params, features, target, test_features, test_target, scorer):
    models_p = pd.DataFrame(columns=['model','g_param','test_time','test_score'])    
    tested_model = pd.DataFrame()
    for i, row in params.iterrows():
        el_time = time.time()
        try:
            model = get_model(row, features, target)    
            predict = model.predict(test_features)
            score = round(scorer(test_target,predict),2) 
        except:
            score = 'Err'
        el_time = round(time.time() - el_time,2)
        disp_font_size(MODEL_TEXT_NAME[row['model']],5)  
        disp_font_size(re.sub(r'[\{\}]', '', str(row['g_param'])),4)  
        display_metric(score, '', title='Тестовая выборка')
        tested_model = row[['model','g_param']]
        tested_model['test_score'] = score
        tested_model['test_time'] = el_time
        models_p = models_p.append(tested_model, ignore_index=True) 
    return models_p.sort_values(by='test_score', ascending=True).reset_index(drop=True)
        
def models_serch(params, features, target, test_features=None, test_target=None, scorer=[f1_score,True]
                 , test=False, test_num=1):
    models_p = pd.DataFrame()   
    
    my_scorer = make_scorer(scorer[0], greater_is_better=scorer[1])  
    for model, params in params.items():
        model_name = re.sub(r".*\_", '', str(model))    
        log = get_model_params(model_name, features, target, my_scorer, disp_metr=True, **params, debug=DEBUGING)
        sort_column = 'score'
        test_num_str = test_num
        log['test_time'] = '-'
        log['test_score'] = '-' 
        if test:
            print('======================')
            disp_font_size("Результаты на тестовой выборке",5)  
            if test_num < len(log): test_num_str = test_num
            else: test_num_str = len(log)
            models_test_p = models_test(log.head(test_num_str+1), features, target, test_features, test_target
                                            , scorer=scorer[0])
            display(models_test_p)                
          
            log.loc[0:test_num,'test_time'] = models_test_p.loc[0:test_num,'test_time']
            log.loc[0:test_num,'test_score'] = models_test_p.loc[0:test_num,'test_score']
            sort_column = 'test_score'      
            
        models_p = models_p.append(log, ignore_index=True,sort=False)   
    result = models_p.loc[~models_p[sort_column].isin(['Err','-'] )]
    return  result.sort_values(by=sort_column, ascending=False).reset_index(drop=True)

def div_features_target(df,targetname):
    """
    Функция для разделения таблицы на признаки и целевой признак
    """
    return df.drop([targetname], axis=1), df[targetname]

## Подготовка данных

In [7]:
try:
    comments = pd.read_csv('/datasets/toxic_comments.csv')
except:
    comments = pd.read_csv('toxic_comments.csv')
display(comments.info())
display(comments.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0


In [8]:
vc = comments['toxic'].value_counts()
print(vc)
disp_font_size(f"Нетоксичнык комментарии: {round(vc[0]/len(comments),3)*100}%")
disp_font_size(f"Токсичнык комментарии: {round(vc[1]/len(comments),3)*100}%")

0    143346
1     16225
Name: toxic, dtype: int64


In [9]:
# comments = comments.head(10000)
lem = WordNetLemmatizer()
nltk.download('wordnet')
print(len(comments))
pbar = tqdm(total = len(comments))
def lemmatize(text):
    """
    Лемматизация
    """
    clr_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    lem_text = []
    slt_text = clr_text.split(' ') 
    for word in slt_text:
        lem_text.append(lem.lemmatize(word)) 
    lem_text = ' '.join(lem_text)    
    pbar.update(1)
    return lem_text.lower()

comments['text'] = pd.DataFrame(comments['text'].values.astype('U'))
comments['lem_text'] = comments['text'].apply(lemmatize)
pbar.close()                   
display(comments.head(5))

159571


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  0%|          | 0/159571 [00:00<?, ?it/s]

,text,toxic,lem_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalism just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,d aww he match this background colour i m seemingly stuck with thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no...",0,more i can t make any real suggestion on improvement i wondered if the section statistic should be later on or a subsection of type of accident i think the reference may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one el...
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that s on


## Обучение

### Мешок слов

In [10]:
train_comments, test_comments = train_test_split(comments, test_size=0.3, random_state=R_STATE)
nltk.download('stopwords') 
stopwords = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 1))

# train_corpus = list(train_comments['lem_text'])
# train_features = count_tf_idf.fit_transform(train_corpus)

train_features = list(train_comments['lem_text'])
train_target = train_comments['toxic']

# test_corpus = list(train_comments['lem_text'])
# test_features = count_tf_idf.fit_transform(test_corpus)
# test_target = train_comments['toxic']

test_features = list(test_comments['lem_text'])
test_target = test_comments['toxic']

In [12]:
models_params_find = True
  
if models_params_find:
    models_param = {
            'LogisticReg':{'solver':'liblinear', 'class_weight':'balanced'},
            'DecTreeCls':{'max_depth': range(85,89,1)},
            'RndForestCls':{'max_depth': range(90,91,1), 'n_estimators': range(1,2,1), 'class_weight':'balanced'},
            'LGBMCls':{'max_depth': range(36,40,1),'n_estimators': range(49,51,1), 
                          'learning_rate': np.arange(0.4,0.5,0.1)}
            }            
    
    bow_model_list = models_serch(models_param, train_features, train_target
                                          , test_features,  test_target
                                          , test=True, test_num=0)
else:    
    bow_model_list = pd.DataFrame({'model' : ['LGBMCls','DecTreeCls','LogisticReg','RndForestReg'],
                                        'g_param' : [{'learning_rate': 0.4, 'max_depth': 36, 'n_estimators': 49
                                                      , 'class_weight':'balanced'},
                                                     {'max_depth': 88}, 
                                                     {'solver':'liblinear', 'class_weight':'balanced'},                                                                                                         
                                                     {'max_depth': 50, 'n_estimators': 1},
                                                    ],
                                           'el_time' : [33.97, 63.84, 8.55, 2.19],   
                                             'score' : [ 0.76,  0.72, 0.74, 0.27],
                                         'test_time' : [14.19, 48.39, 2.69, 1.51],   
                                        'test_score' : [ 0.76,  0.72, 0.76, 0.25]}
                                      )

  0%|          | 0/1 [00:00<?, ?it/s]

,model,g_param,el_time,score
0,LogisticReg,"{'class_weight': 'balanced', 'solver': 'liblinear'}",16.34,0.7387


  0%|          | 0/1 [00:00<?, ?it/s]

,model,g_param,test_time,test_score
0,LogisticReg,"{'class_weight': 'balanced', 'solver': 'liblinear'}",11.8,0.76


  0%|          | 0/4 [00:00<?, ?it/s]

...

,model,g_param,test_time,test_score
0,RndForestCls,"{'class_weight': 'balanced', 'max_depth': 90, 'n_estimators': 1}",8.99,0.29


  0%|          | 0/8 [00:00<?, ?it/s]

,model,g_param,el_time,score
0,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 36, 'n_estimators': 50}",42.27,0.7565
1,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 37, 'n_estimators': 50}",42.09,0.7565
2,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 38, 'n_estimators': 50}",42.03,0.7565
3,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 39, 'n_estimators': 50}",43.83,0.7565
4,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 36, 'n_estimators': 49}",42.22,0.7562
5,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 37, 'n_estimators': 49}",41.36,0.7562
6,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 38, 'n_estimators': 49}",41.98,0.7562
7,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 39, 'n_estimators': 49}",42.47,0.7562


  0%|          | 0/1 [00:00<?, ?it/s]

,model,g_param,test_time,test_score
0,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 36, 'n_estimators': 50}",22.63,0.76


### Би-граммы

In [13]:
# features = count_tf_idf.fit_transform(corpus)
# target = comments['toxic']
# tarin_features, test_features, train_target, test_target = train_test_split(
#         features, target, test_size=0.3, random_state=R_STATE)

In [14]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 2))

train_corpus = list(train_comments['lem_text'])
train_features = count_tf_idf.fit_transform(train_corpus)
train_target = train_comments['toxic']

test_corpus = list(train_comments['lem_text'])
test_features = count_tf_idf.fit_transform(test_corpus)
test_target = train_comments['toxic']

In [15]:
models_params_find = False
  
if models_params_find:
    models_param = {
#             'LogisticReg':{'solver':'liblinear', 'class_weight':'balanced'},
#             'DecTreeCls':{'max_depth': range(80,101,5)},
#             'RndForestCls':{'max_depth': range(10,101,20), 'n_estimators': range(1,2,1)},
            'LGBMCls':{'max_depth': range(60,66,1),'n_estimators': range(65,66,1), 
                          'learning_rate': np.arange(0.4,0.5,0.1)}
            }            
    
    bi_gramm_model_list = models_serch(models_param, tarin_features, train_target
                                          , test_features,  test_target
                                          , test=True, test_num=0)
else:    
    bi_gramm_model_list = pd.DataFrame({'model' : ['LGBMCls','DecTreeCls','LogisticReg','RndForestReg'],
                                        'g_param' : [{'learning_rate': 0.4, 'max_depth': 60, 'n_estimators': 65},
                                                     {'max_depth': 100}, 
                                                     {'solver':'liblinear', 'class_weight':'balanced'},                                                                                                         
                                                     {'max_depth': 100, 'n_estimators': 1},

                                                    ],
                                           'el_time' : [83.77, 557.93, 17.18, 10.83],   
                                             'score' : [ 0.76,   0.72,  0.75 , 0.16],
                                         'test_time' : [37.57, 439.41,  7.51,  8.65],   
                                        'test_score' : [ 0.76,   0.72,  0.76,  0.18]}
                                      )

In [16]:
disp_font_size('Мешок слов',5)
display(bow_model_list.sort_values(by='test_score', ascending=False))

,model,g_param,el_time,score,test_time,test_score
0,LogisticReg,"{'class_weight': 'balanced', 'solver': 'liblinear'}",16.34,0.7387,11.8,0.76
1,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 36, 'n_estimators': 50}",42.27,0.7565,22.63,0.76
2,DecTreeCls,{'max_depth': 88},57.05,0.7140,46.71,0.72
3,RndForestCls,"{'class_weight': 'balanced', 'max_depth': 90, 'n_estimators': 1}",11.80,0.2949,8.99,0.29


In [17]:
disp_font_size('Би-граммы',5)
display(bi_gramm_model_list.sort_values(by='test_score', ascending=False))

,model,g_param,el_time,score,test_time,test_score
0,LGBMCls,"{'learning_rate': 0.4, 'max_depth': 60, 'n_estimators': 65}",83.77,0.76,37.57,0.76
2,LogisticReg,"{'solver': 'liblinear', 'class_weight': 'balanced'}",17.18,0.75,7.51,0.76
1,DecTreeCls,{'max_depth': 100},557.93,0.72,439.41,0.72
3,RndForestReg,"{'max_depth': 100, 'n_estimators': 1}",10.83,0.16,8.65,0.18


## Выводы

Лучшими моделями оказались Логическая регрессия и LGBM. Логическая регрессия показала большую скорость.
Добавление би-шрамм в данной задаче не дало увеличение метрики.